In [2]:
import pandas as pd
import os
import numpy as np

In [3]:
path = "C:/Users/doosti/Dropbox (Chapman)/Research/Research Projects/Fitness/Data/"
file = "labeled_comments.csv"

In [32]:
comments_labeled = pd.read_csv(os.path.join(path, file))
comments = pd.read_csv(os.path.join(path, "merged_comments.csv"))
videos = pd.read_csv(os.path.join(path, "videos_101223.csv"))
videos_trends = pd.read_csv(os.path.join(path, "final_yoga_videos_tends_101023.csv"))
trends = pd.read_csv(os.path.join(path, "final_yoga_channels_trends_091523.csv"))
# matching with the creator id
#comments['creator_id'] = comments.merge(videos,how='left',on='video_id').creator_id.values
#comments_labeled['creator_id'] = comments_labeled.merge(videos,how='left',on='video_id').creator_id.values

C:\Users\doosti\AppData\Local\Temp\ipykernel_357096\3566640832.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  comments_labeled = pd.read_csv(os.path.join(path, file))


In [35]:
print(videos.columns)

Index(['video_id', 'title', 'creator', 'creator_id', 'publish_date',
       'duration', 'v1', 'v2', 'v3', 'v7', 'v30', 'e1', 'e2', 'e3', 'e7',
       'e30', 'views', 'enagement', 'likes', 'shares', 'comments', 'keywords',
       'topics', 'challenge'],
      dtype='object')


In [36]:
print(videos_trends.head())

      video_id        date   views  engagements
0  CpoaI_rVRXU  2023-09-06  2023.0        100.0
1  CpoaI_rVRXU  2023-09-07   918.0          5.0
2  CpoaI_rVRXU  2023-09-08    45.0          1.0
3  CpoaI_rVRXU  2023-09-09    33.0          0.0
4  CpoaI_rVRXU  2023-09-10    36.0          0.0


In [37]:
print(comments.head())

      video_id                   user_id  \
0  YxySUy2jU_s  UCLqs5JoktNvfrQZimZ8Gl5g   
1  YxySUy2jU_s  UCLqs5JoktNvfrQZimZ8Gl5g   
2  YxySUy2jU_s  UCLMqrKUz0vnpsM0haZmHmQA   
3  YxySUy2jU_s  UCASrUw6lnbPK4p5plkVFtqw   
4  YxySUy2jU_s  UCspkSCEGRikNmsoRGJwG1MQ   

                                        comment_text  no_likes  no_replies  \
0  I loved doing this video with my dad I think i...       8.0         0.0   
1             bre is very good at yoga same with Flo      10.0         0.0   
2  Of course I can add 15 more minutes to round o...       7.0         0.0   
3  Thanks for helping round out my routine, was q...       5.0         0.0   
4          First thing tomorrow morning! Thank you 🙏       2.0         0.0   

   comment_year  comment_month  comment_day  
0        2021.0            4.0         13.0  
1        2021.0            4.0         13.0  
2        2021.0            4.0         13.0  
3        2021.0            4.0         13.0  
4        2021.0            4.0      

In [40]:
# threshold = 11
comments_labeled = comments_labeled[comments_labeled.video_id.str.len() == 11].copy()
print(comments_labeled.shape)
# drop rows with missing year, month, or day
comments_labeled = comments_labeled.dropna(subset=['comment_year', 'comment_month', 'comment_day'])
print(comments_labeled.shape)
print(comments_labeled.isnull().sum())

(827566, 10)
(827552, 10)
video_id              0
user_id               0
comment_text        100
no_likes              0
no_replies            0
comment_year          0
comment_month         0
comment_day           0
label            764497
label_score      764497
dtype: int64


In [41]:
# Create a function to format and convert the date
def to_datetime_with_padding(year, month, day):
    # Pad month and day with zeros if necessary
    month = f"{month:02d}"
    day = f"{day:02d}"
    # Combine and convert to datetime
    date_str = f"{year}-{month}-{day}"
    return pd.to_datetime(date_str)

comments_labeled['comment_year'] = comments_labeled.comment_year.astype(int)
comments_labeled['comment_month'] = comments_labeled.comment_month.astype(int)
comments_labeled['comment_day'] = comments_labeled.comment_day.astype(int)

# Apply the function to each row
comments_labeled['date'] = comments_labeled.apply(lambda row: to_datetime_with_padding(row["comment_year"], row["comment_month"], row["comment_day"]), axis=1)

In [42]:
# merging with video dataset
#comments_labeled = 
comments_labeled['creator_id'] = comments_labeled.merge(videos,how='left',on='video_id').creator_id.values
print(comments_labeled.isnull().sum())

channel_day = comments_labeled.fillna(0).groupby(['creator_id','date']).label.value_counts().unstack().reset_index()
channel_day = channel_day.fillna(0)
print(channel_day.head())


   video_id  user_id  comment_text  no_likes  no_replies  comment_year  \
0     False    False         False     False       False         False   
1     False    False         False     False       False         False   
2     False    False         False     False       False         False   
3     False    False         False     False       False         False   
4     False    False         False     False       False         False   

   comment_month  comment_day  label  label_score   date  creator_id  
0          False        False   True         True  False       False  
1          False        False   True         True  False       False  
2          False        False   True         True  False       False  
3          False        False   True         True  False       False  
4          False        False   True         True  False       False  
label creator_id       date      0  appearance  competence  fitness  interest  \
0              0 2020-09-10    1.0         0.0  

In [45]:
channel_day.to_csv(os.path.join(path, "channel_day.csv"), index=False)

In [47]:
channel_day = pd.read_csv(os.path.join(path, "channel_day.csv"))
merged = trends.merge(channel_day,how='left',on=['creator_id','date']) #.to_csv(os.path.join(path, "trends_channel_day.csv"), index=False)
merged.isnull().sum()
print(f"number of creators with comments: {merged[merged.appearance.notnull()].creator_id.nunique()}, out of {merged.creator_id.nunique()} creators in trends")
# there seems to be a problem with the data set, the number of creators with comments is less than the number of creators in trends

number of creators with comments: 71, out of 75 creators in trends


In [19]:
# check if the issue is with "date"
merged = trends.merge(channel_day,how='left',on=['creator_id'])
print(merged[merged.appearance.notnull()].creator_id.nunique())
# no, the issue is not with "date"

23


## Individual Level Data

In [52]:
comments_labeled.groupby(['user_id','date']).comment_text.count().reset_index().to_csv(os.path.join(path, "user_day.csv"), index=False)